In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack, Column, join, MaskedColumn
from astropy import units as u
from astroquery.simbad import Simbad
from astroquery.gaia import Gaia
import time

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
sample = Table.read('Gr8stars_GaiaeDR3_TIC_hires.csv')
sample.sort(['_RAJ2000','_DEJ2000'])

In [3]:
table = sample['_RAJ2000', '_DEJ2000']
table.rename_column('_RAJ2000', 'ra_j2000')
table.rename_column('_DEJ2000', 'dec_j2000')

In [4]:
# names:
table['primary_name'] = sample['EDR3Name'] # CHANGE THIS LATER
table['gaiadr2_source_id'] = sample['GAIA']
table['gaiaedr3_source_id'] = sample['Source']
table['hip'] = sample['HIP']
table.add_column(Column(name='hd', dtype='<U29', length=len(table)))
table['tic'] = sample['TIC']
table['2mass'] = sample['2MASS']
table['wisea'] = sample['WISEA']

In [5]:
# gaia:
table['ra'] = sample['RA_ICRS']
table['ra_error'] = sample['e_RA_ICRS']
table['dec'] = sample['DE_ICRS']
table['dec_error'] = sample['e_DE_ICRS']
table['parallax'] = sample['Plx']
table['parallax_error'] = sample['e_Plx']
table['pmra'] = sample['pmRA']
table['pmra_error'] = sample['e_pmRA']
table['pmdec'] = sample['pmDE']
table['pmdec_error'] = sample['e_pmDE']
table['gmag'] = sample['Gmag']
table['bp_rp'] = sample['BPRP']
table['radial_velocity'] = sample['RVDR2']
table['radial_velocity_error'] = sample['e_RVDR2']
table['ref_epoch'] = 'J2016.0'

### add SIMBAD alternative identifiers:

In [6]:
# correct some failed matches by hand (ugh):
# for most of these the TIC x-match failed, therefore SIMBAD does not include the TIC ID
# in three cases there is no Gaia DR2 ID so HIP, 2MASS, WISEA must also be copied as SIMBAD query on DR2 ID will fail
table['tic'][table['gaiaedr3_source_id'] == 411413822074322432] = 252191219
table['gaiadr2_source_id'][table['gaiaedr3_source_id'] == 411413822074322432] = 411413822074322432
table['tic'][table['gaiaedr3_source_id'] == 883596170039715072] = 101637926
table['gaiadr2_source_id'][table['gaiaedr3_source_id'] == 883596170039715072] = 883596170039715072
table['tic'][table['gaiaedr3_source_id'] == 1030087329727496320] = 459753450
table['hip'][table['gaiaedr3_source_id'] == 1030087329727496320] = 43630  # from TIC
table['2mass'][table['gaiaedr3_source_id'] == 1030087329727496320] = '08530805+5457112'  # from TIC
table['wisea'][table['gaiaedr3_source_id'] == 1030087329727496320] = 'J085308.10+545711.5'  # from TIC
table['tic'][table['gaiaedr3_source_id'] == 3890250298609911168] = 63189338
table['hip'][table['gaiaedr3_source_id'] == 3890250298609911168] = 51145  # from TIC
table['2mass'][table['gaiaedr3_source_id'] == 3890250298609911168] = '10265302+1713099'  # from TIC
table['wisea'][table['gaiaedr3_source_id'] == 3890250298609911168] = 'J102652.99+171309.1'  # from TIC
table['tic'][table['gaiaedr3_source_id'] == 762815470562110464] = 166646191 # this one is ALSO TIC 353969903
table['hip'][table['gaiaedr3_source_id'] == 762815470562110464] = 54035  # from TIC
table['2mass'][table['gaiaedr3_source_id'] == 762815470562110464] = '11032027+3558203'  # from TIC
table['wisea'][table['gaiaedr3_source_id'] == 762815470562110464] = 'J110319.67+355722.4'  # from SIMBAD
table['tic'][table['gaiaedr3_source_id'] == 778947814402602752] = 252803603
table['gaiadr2_source_id'][table['gaiaedr3_source_id'] == 778947814402602752] = 778947814402405120
table['tic'][table['gaiaedr3_source_id'] == 4445793563751403264] = 276883893 # magnitude changed considerably DR2 to eDR3 hence failed checks but seems ok to me
table['gaiadr2_source_id'][table['gaiaedr3_source_id'] == 4445793563751403264] = 4445793563749783424
table['tic'][table['gaiaedr3_source_id'] == 4468557611984384512] = 398120047
table['gaiadr2_source_id'][table['gaiaedr3_source_id'] == 4468557611984384512] = 4468557611977678336

In [7]:
# more failed matches by hand:
# these all have correct TIC x-match but TIC and DR2 IDs are not ingested properly in SIMBAD
# so we'll query on an alternative identifier
weird_ones_map = {'Gaia DR2 386648181647057152':'HD 225291A',
                 'Gaia DR2 2551241944269355776': 'HD 3972A',
                 'Gaia DR2 417565761431676288': 'HIP 3585',
                 'Gaia DR2 2572433347264096768': 'HIP 8110',
                 'Gaia DR2 90783037268826368': 'HIP 9434',  # bright double
                 'Gaia DR2 139281533098990336': 'HD 19771',  # bright double
                 'Gaia DR2 450345089271313536': 'HD 21903',  # bright double
                 'Gaia DR2 2989285658784560640': 'HIP 23951',  # bright double
                 'Gaia DR2 3235618049144649984': 'HD 33866',
                 'Gaia DR2 3319596479531147392': 'HD 38767',  # bright double
                 'Gaia DR2 3049638303010222080': 'HD 52590',
                 'Gaia DR2 901953203859813888': 'HD 67501A',  # co-moving twin pair?
                 'Gaia DR2 901953199567368960': 'HD 67501B',  # co-moving twin pair?
                 'Gaia DR2 903616971110812544': 'HD 71974',  # bright double
                 'Gaia DR2 811174736439695232': 'HD 80441', # bright double
                 'Gaia DR2 3850860191466194688': 'HD 82267',
                 'Gaia DR2 1070274204844445056': 'HIP 49230',
                 'Gaia DR2 746691643641649536': 'HIP 49315',
                 'Gaia DR2 4022691525057422208': 'HIP 56054',
                 'Gaia DR2 1466569467151572992': 'HD 114723',  # bright double ?
                 'Gaia DR2 1603382317472430080': 'HIP 71109',
                 'Gaia DR2 1186325496788069632': 'HD 131473',  # bright double
                 'Gaia DR2 4409032079673958528': 'HIP 78024',
                 'Gaia DR2 1191393008443719808': 'TYC 956-1098-1',  # HD 143597. no survey other than Tycho and Gaia resolves them
                 'Gaia DR2 1191393008441035648': 'TYC 956-1098-2',  # HD 143597
                 'Gaia DR2 4148904396147803136': 'HD 162739',  # duplicated in TIC as TIC 207281346
                 'Gaia DR2 4185205008718146048': 'HD 179518',  # this HD is TIC as 6990809 with a different DR2 ID but no Gaia info, diff V mag ??
                 'Gaia DR2 4516054074788081152': 'HD 180054',  # duplicated in TIC as TIC 352587633
                 'Gaia DR2 2087620020176419072': 'HIP 99388',
                 'Gaia DR2 1804783160869679872': 'HIP 101315',
                 'Gaia DR2 1731257443928035200': 'HIP 104047',
                 'Gaia DR2 1787693589076214016': 'HD 205160B',  # associated twin stars w slightly different PMRA?
                 'Gaia DR2 1787693589076213632': 'HD 205160A',  # associated twin stars w slightly different PMRA?
                 'Gaia DR2 2681975358571981312': 'HD 209965',  # bright double
                 'Gaia DR2 1929623635232510208': 'HIP 112722',
                 'Gaia DR2 2643881159024617088': 'HIP 115891'}

In [8]:
# hacky alternative to rerunning all the simbad code:
table2 = Table.read('Gr8stars_GaiaeDR3_formatted-names.csv')
table2.sort(['ra_j2000','dec_j2000'])
table['hd'] = table2['hd']
table['primary_name'] = table2['primary_name']
table['hip'] = table2['hip']
table['2mass'] = table2['2mass']
table['wisea'] = table2['wisea']

In [9]:
table['hip'] = MaskedColumn(table['hip'], mask=(table['hip'] <= 0))

In [10]:
table[0:10]

ra_j2000,dec_j2000,primary_name,gaiadr2_source_id,gaiaedr3_source_id,hip,hd,tic,2mass,wisea,ra,ra_error,dec,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,gmag,bp_rp,radial_velocity,radial_velocity_error,ref_epoch
float64,float64,str29,int64,int64,int64,str7,int64,str16,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str7
0.1735187163099,62.1758995053047,HD224792,429915991435184000,429915991435184000,58,224792,417860263,00004167+6210331,J000041.58+621032.9,0.17307161161,0.0135,62.17570631925,0.0153,26.1048,0.0182,-46.955,0.017,-43.467,0.019,6.922204,0.67424,3.76,0.27,J2016.0
0.2923327649201,-0.0761104076889,HD224839,2546019543207725440,2546019543207725440,93,224839,138684373,00011015-0004338,J000110.18-000434.6,0.29252029523,0.0274,-0.07639817491,0.0198,15.7099,0.0311,42.194,0.038,-64.748,0.019,7.981884,0.764665,-3.02,0.16,J2016.0
0.4571711894795,65.4555699380329,HD224919,432197890377254528,432197890377254528,--,224919,418988281,00014974+6527200,J000149.74+652720.6,0.45736153541,0.0117,65.45580468513,0.012,15.3692,0.0151,17.791,0.015,52.818,0.016,7.535949,0.631671,-29.18,0.15,J2016.0
0.5431196625046,27.0818330846237,85 Peg,2853900872207150720,2853900872207888768,171,224930,611726941,00021014+2704570,J000210.87+270444.6,0.54669605643,0.2633,27.07764869795,0.2213,81.179,0.449,716.46,0.452,-941.497,0.242,5.595876,0.929018,-35.57,0.35,J2016.0
0.6965612550145,2.1301527341538,HD225028A,2739059700184753280,2739059700184753280,--,225028A,611064061,--,--,0.69683851342,0.0193,2.12973783087,0.0154,23.3713,0.0233,62.34,0.028,-93.353,0.015,7.213268,0.799183,-2.54,0.19,J2016.0
0.8580574171151,20.6655286255845,HD225097,2846469822870413696,2846469822870413696,276,225097,238301314,00032593+2039561,J000325.97+203954.3,0.85833620764,0.144,20.66483347784,0.185,14.5756,0.225,58.692,0.187,-156.408,0.236,7.341593,0.79231,-31.51,0.4,J2016.0
1.2346725514673,23.2696293916025,HD225261,2848150288954631296,2848150288954631296,400,225261,258866681,00045634+2316106,J000456.61+231610.6,1.23652388841,0.0188,23.26959631737,0.0219,37.9297,0.0258,382.667,0.024,-7.444,0.029,7.624367,0.940167,7.51,0.16,J2016.0
1.2396804613202,45.6737925276659,HD225291A,386648181647057152,386648181647057152,404,225291A,439946040,00045752+4540257,J000457.52+454025.5,1.23979666771,0.0154,45.67350137279,0.012,13.6393,0.0222,18.27,0.019,-65.51,0.016,7.438472,0.694592,-17.15,0.16,J2016.0
1.3798428459156,-9.6174575097273,HD63,2428706669045383808,2428706669045383808,462,63,176285709,00053115-0937028,J000531.22-093703.7,1.38022917866,0.0325,-9.61779490682,0.0212,20.1669,0.036,85.703,0.041,-75.914,0.024,7.020907,0.603799,0.44,0.31,J2016.0


### add Gaia DR2:

In [11]:
# gaia DR2:
table['primary_name','gaiadr2_source_id'].write('gaiadr2.vot', format='votable', overwrite=True)  # write out the DR2 source_ids
#Gaia.ROW_LIMIT = 5000
query = "SELECT TOP 5000 * FROM tap_upload.tbl AS tbl, gaiadr2.gaia_source AS gaia WHERE tbl.gaiadr2_source_id=gaia.source_id"
j = Gaia.launch_job(query=query, upload_resource='gaiadr2.vot', 
                    upload_table_name="tbl", verbose=True)
r = j.get_results()

Launched query: 'SELECT TOP 5000 * FROM tap_upload.tbl AS tbl, gaiadr2.gaia_source AS gaia WHERE tbl.gaiadr2_source_id=gaia.source_id'
------>https
host = gea.esac.esa.int:443
context = /tap-server/tap/sync
Content-type = multipart/form-data; boundary====1626796662816===
200 200
[('Date', 'Tue, 20 Jul 2021 15:57:43 GMT'), ('Server', 'Apache/2.4.6 (CentOS) OpenSSL/1.0.2k-fips PHP/5.4.16 mod_jk/1.2.43'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=block'), ('X-Frame-Options', 'SAMEORIGIN'), ('X-Content-Type-Options', 'nosniff'), ('Set-Cookie', 'JSESSIONID=E82EC7A91AA63C7FE751969336349660; Path=/tap-server; Secure; HttpOnly'), ('Content-Encoding', 'gzip'), ('Content-Disposition', 'attachment;filename="1626796663335O-result.vot"'), ('Transfer-Encoding', 'chunked'), ('Content-Type', 'application/x-votable+xml')]
Retrieving sync. results...
Query finished.


In [12]:
r.columns

<TableColumns names=('primary_name','gaiadr2_source_id','solution_id','designation','source_id','random_index','ref_epoch','ra','ra_error','dec','dec_error','parallax','parallax_error','parallax_over_error','pmra','pmra_error','pmdec','pmdec_error','ra_dec_corr','ra_parallax_corr','ra_pmra_corr','ra_pmdec_corr','dec_parallax_corr','dec_pmra_corr','dec_pmdec_corr','parallax_pmra_corr','parallax_pmdec_corr','pmra_pmdec_corr','astrometric_n_obs_al','astrometric_n_obs_ac','astrometric_n_good_obs_al','astrometric_n_bad_obs_al','astrometric_gof_al','astrometric_chi2_al','astrometric_excess_noise','astrometric_excess_noise_sig','astrometric_params_solved','astrometric_primary_flag','astrometric_weight_al','astrometric_pseudo_colour','astrometric_pseudo_colour_error','mean_varpi_factor_al','astrometric_matched_observations','visibility_periods_used','astrometric_sigma5d_max','frame_rotator_object_type','matched_observations','duplicated_source','phot_g_n_obs','phot_g_mean_flux','phot_g_mean_fl

In [13]:
dr2_table = r['gaiadr2_source_id', 'teff_val']
dr2_table.rename_column('teff_val', 'gaia_teff')
dr2_table['gaia_teff_lower'] = r['teff_percentile_lower']
dr2_table['gaia_teff_upper'] = r['teff_percentile_upper']
dr2_table['gaia_radius'] = r['radius_val']
dr2_table['gaia_radius_lower'] = r['radius_percentile_lower']
dr2_table['gaia_radius_upper'] = r['radius_percentile_upper']
dr2_table['gaia_lum'] = r['lum_val']
dr2_table['gaia_lum_lower'] = r['lum_percentile_lower']
dr2_table['gaia_lum_upper'] = r['lum_percentile_upper']

In [14]:
table_with_dr2 = join(table, dr2_table, keys='gaiadr2_source_id', join_type='left')

In [15]:
assert len(table_with_dr2) == len(table)

In [16]:
table_with_dr2.sort(['ra_j2000','dec_j2000']) # get it in the same order as sample
for i in np.random.randint(0,len(table),42):
    #print(table[i], table_with_dr2[i])
    assert table[i]['gaiadr2_source_id'] == table_with_dr2[i]['gaiadr2_source_id']

In [17]:
table = table_with_dr2

### add hires info:

In [18]:
table.add_column(Column(name='spectrum_hires', dtype='<U29', length=len(table)))

In [19]:
# spectra:
hires_table = Table()
hires_table.add_column(Column(name='gaiaedr3_source_id', dtype=np.int64, length=len(sample)))
for i,s in enumerate(sample['EDR3Name']):
    hires_table[i]['gaiaedr3_source_id'] = s.split()[2] # hack relying on spaces
hires_table['spectrum_hires_resolution'] = MaskedColumn(sample['hires_maxres'], 
                                                  mask=(sample['hires_maxres'] == 0))
hires_table['spectrum_hires_nobs'] = MaskedColumn(sample['n_hires_obs_maxres_iodout'], 
                                            mask=(sample['n_hires_obs_maxres_iodout'] == 0))
hires_table['spectrum_hires_snr'] = MaskedColumn(sample['hires_maxres_snr'], 
                                           mask=(sample['hires_maxres_snr'] == 0))

In [20]:
table_with_hires = join(table, hires_table, keys='gaiaedr3_source_id', join_type='left')

In [22]:
# add a link:
MIN_SNR = 100 # if combined SNR is below this limit, ignore the spectra
bad = table_with_hires['spectrum_hires_snr'] < MIN_SNR
table_with_hires['spectrum_hires_resolution'][bad] = np.ma.masked
table_with_hires['spectrum_hires_nobs'][bad] = np.ma.masked
table_with_hires['spectrum_hires_snr'][bad] = np.ma.masked
table_with_hires['spectrum_hires'][~table_with_hires['spectrum_hires_snr'].mask] = 'true'

In [28]:
table = table_with_hires

### add harps info:

In [29]:
harps_table = Table.read('annelies/Gr8_spectra_info_HARPS.txt', format='ascii', comment='-')

In [30]:
harps_table.rename_column('Name', 'primary_name')
harps_table.rename_column('S2D HARPS', 'spectrum_harps_nobs')
harps_table.rename_column('SNR co-added HARPS', 'spectrum_harps_snr')

In [31]:
harps_table[harps_table['primary_name'] == '111 Tau B']

primary_name,spectrum_harps_nobs,spectrum_harps_snr
str9,int64,float64
111 Tau B,7,7.9


In [32]:
table.add_column(Column(name='spectrum_harps', dtype='<U29', length=len(table)))

In [33]:
table_with_harps = join(table, harps_table, keys='primary_name', join_type='left')

In [34]:
table_with_harps['spectrum_harps_nobs'].fill_value = 0
table_with_harps['spectrum_harps_snr'].fill_value = 0.

In [36]:
table_with_harps['spectrum_harps'][(table_with_harps['spectrum_harps_nobs'] > 0) 
                                   & ~(table_with_harps['spectrum_harps_nobs'].mask)] = 'true'

In [37]:
table = table_with_harps

In [38]:
table[9]

ra_j2000,dec_j2000,primary_name,gaiadr2_source_id,gaiaedr3_source_id,hip,hd,tic,2mass,wisea,ra,ra_error,dec,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,gmag,bp_rp,radial_velocity,radial_velocity_error,ref_epoch,gaia_teff,gaia_teff_lower,gaia_teff_upper,gaia_radius,gaia_radius_lower,gaia_radius_upper,gaia_lum,gaia_lum_lower,gaia_lum_upper,spectrum_hires,spectrum_hires_resolution,spectrum_hires_nobs,spectrum_hires_snr,spectrum_harps,spectrum_harps_nobs,spectrum_harps_snr
,,,,,,,,,,,,,,,,,,,,,,,,,K,K,K,Rsun,Rsun,Rsun,Lsun,Lsun,Lsun,,,,,,,
float64,float64,str29,int64,int64,int64,str7,int64,str16,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str7,float32,float32,float32,float32,float32,float32,float32,float32,float32,str29,float64,int64,float64,str29,int64,float64
80.9099146163007,17.324116905683,111 Tau B,3394298532176344960,3394298532176344960,25220,35171,47345026,05233836+1719267,J052338.57+171926.8,80.91108310812,0.02,17.32409154335,0.0139,68.7662,0.0246,250.984,0.027,-5.707,0.019,7.543094,1.348195,37.94,0.13,J2016.0,4628.6665,4576.0,4712.0,0.6723673,0.6487954,0.68793327,0.18695256,0.18674259,0.18716253,,--,--,--,true,7,7.9


### write it out:

In [39]:
table[:10]

ra_j2000,dec_j2000,primary_name,gaiadr2_source_id,gaiaedr3_source_id,hip,hd,tic,2mass,wisea,ra,ra_error,dec,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,gmag,bp_rp,radial_velocity,radial_velocity_error,ref_epoch,gaia_teff,gaia_teff_lower,gaia_teff_upper,gaia_radius,gaia_radius_lower,gaia_radius_upper,gaia_lum,gaia_lum_lower,gaia_lum_upper,spectrum_hires,spectrum_hires_resolution,spectrum_hires_nobs,spectrum_hires_snr,spectrum_harps,spectrum_harps_nobs,spectrum_harps_snr
,,,,,,,,,,,,,,,,,,,,,,,,,K,K,K,Rsun,Rsun,Rsun,Lsun,Lsun,Lsun,,,,,,,
float64,float64,str29,int64,int64,int64,str7,int64,str16,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str7,float32,float32,float32,float32,float32,float32,float32,float32,float32,str29,float64,int64,float64,str29,int64,float64
238.1689210400862,42.4515174680292,1 Her,1384195086792268800,1384195086792268800,77760,142373,157364190,15524052+4227052,J155240.95+422712.1,238.17156439238,0.0608,42.45431545672,0.0739,62.9028,0.0807,438.839,0.08,629.541,0.099,4.424265,0.784468,-56.01,0.16,J2016.0,6073.0,5888.5,6158.0,1.6416837,1.5966756,1.7461708,3.3028345,3.2885118,3.3171573,,--,--,--,,--,--
126.1458923835362,-3.7512424989997,1 Hya,3066710282611675776,3066710282611675776,41211,70958,168975758,08243502-0345045,J082434.89-034504.6,126.14494794137,0.045,-3.75135791807,0.032,36.9674,0.061,-212.044,0.06,-25.969,0.059,5.468879,0.648508,96.87,5.64,J2016.0,6320.5,6246.0,6395.0,1.5511353,1.5152051,1.5883586,3.459392,3.4467092,3.472075,,--,--,--,,--,--
191.2475206828197,39.2789161469845,10 CVn,1521686782461765888,1521686782461765888,62207,110897,389853353,12445938+3916441,--,191.24545542461,0.0197,39.27953401499,0.0174,56.9588,0.0323,-359.699,0.018,139.016,0.018,5.803025,0.756415,80.45,0.19,J2016.0,5967.75,5926.5,6026.3335,0.98277783,0.96376306,0.9965062,1.103692,1.1017659,1.1056182,true,102700.0,3,139.7855500400524,,--,--
54.2182654350701,0.4016644716193,10 Tau,3263836568393943808,3263836568394170880,16852,22484,649766486,03365238+0024054,J033652.23+002400.9,54.21723179469,0.1306,0.39952459488,0.1093,71.837,0.1503,-232.563,0.164,-481.472,0.143,4.141093,0.757795,nan,nan,J2016.0,6082.3335,5830.0,6164.6665,--,--,--,--,--,--,true,71600.0,2,151.42654985173505,,--,--
74.9346751253041,15.9167324100169,101 Tau,3393284752392701312,3393284752392701312,23214,31845,389121767,04594432+1555002,J045944.38+155459.9,74.93506827868,0.0241,15.91660060912,0.0127,22.3771,0.0262,85.068,0.033,-29.655,0.021,6.644502,0.59449,41.72,0.54,J2016.0,6520.25,6444.5,6551.5,1.4123555,1.398914,1.4457529,3.2481775,3.2326443,3.2637107,,--,--,--,true,1,105.9
76.8625232015097,18.6450499366847,104 Tau,3407611182744289152,3407611182744289152,23835,32923,27136704,05072692+1838420,--,76.86503115411,0.0897,18.64513194789,0.0566,62.8252,0.1094,534.674,0.111,18.449,0.083,4.732457,0.844065,20.58,0.15,J2016.0,5835.0,5700.0,6022.0,1.5503769,1.4555848,1.6246854,2.510341,2.4968112,2.5238707,true,71600.0,3,196.42810389554748,true,48,394.1
315.1410034667797,-4.7302596237905,11 Aqr,6912276699251278848,6912276699251278848,103682,199960,248854510,21003382-0443486,J210033.85-044350.1,315.1412210745,0.0209,-4.73087375776,0.0137,36.9971,0.0293,48.795,0.028,-138.18,0.021,6.063653,0.779556,-17.34,0.14,J2016.0,5920.0,5863.0,5967.0,1.3689218,1.3474416,1.3956684,2.0736659,2.069347,2.0779848,,--,--,--,true,77,208.8
143.9145924739025,35.8101325142152,11 LMi,798068905726303232,798068905726303232,47080,82885,8915802,09353959+3548366,J093538.92+354834.4,143.91061083593,0.0692,35.80898121341,0.0601,89.0092,0.0937,-726.514,0.07,-259.057,0.062,5.200431,0.934127,14.39,0.17,J2016.0,5661.0,5488.0,6794.75,0.93610704,0.64977777,0.99605566,0.8108086,0.8083409,0.8132763,,--,--,--,,--,--


In [40]:
from astropy.io import ascii
ascii.write(table, 'Gr8stars_GaiaeDR3_formatted.csv', delimiter=',', formats={'spectrum_hires_snr':'%.1f'},
           comment=False, overwrite=True)